# Transform into a DataFrame logs data.


In [40]:
import os
import json
import pandas as pd

# Directory where your JSON log files are stored.
CURRENT_DIR = os.getcwd()
LOGS_DIR = os.path.join(CURRENT_DIR, "dataset/logs")

# Lists to store extracted game-level and round-level data.
games_data = []
choices_data = []

# Iterate over all files in the logs folder.
for filename in os.listdir(LOGS_DIR):
    if filename.endswith('.json'):
        file_path = os.path.join(LOGS_DIR, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Parse game-level data.
        game_id = data.get("game_id")
        start_time = pd.to_datetime(data.get("start_time"), utc=True)
        completion_time = pd.to_datetime(data.get("completion_time"), utc=True) if data.get("completion_time") else None
        total_duration = data.get("total_duration")
        success = data.get("success")
        
        
        # Retrieve final_choice: either as a top-level key or from within choices.
        final_choice_data = data.get("final_choice")
        if not final_choice_data:
            # Look for an entry in choices with "type": "final_choice".
            for choice in data.get("choices", []):
                if choice.get("type") == "final_choice":
                    final_choice_data = choice
                    break

        if final_choice_data:
            chosen_quadrant = final_choice_data.get("chosen_quadrant")
            correct = final_choice_data.get("correct")
            score = final_choice_data.get("score")
            biased_quadrant = final_choice_data.get("biased_quadrant")
        else:
            chosen_quadrant = None
            correct = None
            score = None
            biased_quadrant = None
            
            
        if "total_duration" not in data or "completion_time" not in data or "success" not in data or "start_time" not in data or "game_id" not in data or not final_choice_data or not data.get("choices"):
            print(filename)
            continue
        
        games_data.append({
            "game_id": game_id,
            "start_time": start_time,
            "completion_time": completion_time,
            "total_duration": total_duration,
            "success": success,
            "chosen_quadrant": chosen_quadrant,
            "correct": correct,
            "score": score,
            "biased_quadrant": biased_quadrant,
            "source_file": filename  # Track the source file if needed.
        })
        
        # Process each entry in the choices list.
        for choice in data.get("choices", []):
            # Skip the final_choice entry if it was provided inside choices.
            if choice.get("type") == "final_choice":
                continue

            round_num = choice.get("round")
            quadrant = choice.get("quadrant")
            # Use "cue_name" if available; otherwise fallback to "choice".
            cue_name = choice.get("cue_name") if "cue_name" in choice else choice.get("choice")
            color = choice.get("color")
            timestamp = pd.to_datetime(choice.get("timestamp"), utc=True) if choice.get("timestamp") else None
            # Optional: if client_timestamp is provided.
            client_timestamp = pd.to_datetime(choice.get("client_timestamp"), utc=True) if choice.get("client_timestamp") else None
            # choice_number may not exist in all formats.
            choice_number = choice.get("choice_number")
            
            choices_data.append({
                "game_id": game_id,
                "round": round_num,
                "quadrant": quadrant,
                "cue_name": cue_name,
                "color": color,
                "timestamp": timestamp,
                "client_timestamp": client_timestamp,
                "choice_number": choice_number,
                "source_file": filename  # Track the source file.
            })

# Convert the lists to pandas DataFrames for further analysis.
games_df = pd.DataFrame(games_data)
choices_df = pd.DataFrame(choices_data)

print("Choices DataFrame:")
display(choices_df)

print("Games DataFrame:")
display(games_df.sort_values(by="total_duration", ascending=False))

game_fe328370-fc7a-42c6-badc-1064aa664a72.json
game_ffe9f3c9-9d17-4e01-94c0-882f12d21d10.json
game_fdf5bea9-1871-4120-95d1-c8694fa7f9dc.json
game_fff7b9de-c9c7-484c-8140-3227ddc146bb.json
game_f7fbf02f-8936-40e5-8314-83ccfb18fac9_20250204_122105.json
game_ff71b176-8967-44e6-bc6c-34be84754240.json
game_fd05ec48-ea8d-429a-8db8-64026587f9bb.json
game_fe07765b-29c1-4e17-b21f-0daab3875971.json
game_fd32ebee-d251-4e48-8991-bf0759abedad.json
game_ff8ffca6-ccf8-4c82-a4fa-8e9cfb1ff267.json
game_ffc0707e-f723-41a7-937d-39c1bd56d22f.json
game_fd8be829-b4d8-4193-b749-a854dc14e151.json
game_fe99619d-646c-47e0-8876-dea3e9f7286b.json
game_fce2ecac-9ef1-498d-aa95-f7c9760f26ae.json
game_fd50c40d-6d43-403b-9f91-a3f7525eeec2.json
game_ff312307-84e8-43ed-a24f-b3961aaa23ee.json
game_ff5c4af9-3fc1-4f25-aae2-bb2a362f1112.json
game_9d8faf55-3f6f-4b79-8fb4-fe7f99d9d00e_20250204_124407.json
game_ffbfca36-a0d8-43a4-a737-b2f9771587d9.json
game_fee0a9f8-03f7-4a84-9e67-7b60cb1a2e70.json
game_fcbf47d6-f972-40f7-9c14

,game_id,round,quadrant,cue_name,color,timestamp,client_timestamp,choice_number,source_file
0,153f891f-9273-4bc4-a0d6-f78e8f6981df,0,0,A,RED,2025-02-13 13:26:31.236000+00:00,None,1,game_153f891f-9273-4bc4-a0d6-f78e8f6981df.json
1,153f891f-9273-4bc4-a0d6-f78e8f6981df,1,0,A,RED,2025-02-13 13:26:32.951000+00:00,None,1,game_153f891f-9273-4bc4-a0d6-f78e8f6981df.json
2,153f891f-9273-4bc4-a0d6-f78e8f6981df,2,0,A,RED,2025-02-13 13:26:34.484000+00:00,None,1,game_153f891f-9273-4bc4-a0d6-f78e8f6981df.json
3,f32696f2-9de2-4678-8493-0afe7b692f10,0,0,A,GREEN,2025-02-13 13:42:55.232000+00:00,None,1,game_f32696f2-9de2-4678-8493-0afe7b692f10.json
4,f32696f2-9de2-4678-8493-0afe7b692f10,1,1,B,RED,2025-02-13 13:42:58.303000+00:00,None,1,game_f32696f2-9de2-4678-8493-0afe7b692f10.json
...,...,...,...,...,...,...,...,...,...
903,dc7af284-a056-4a87-bfe0-bd20f87fada9,9,1,B,GREEN,2025-02-11 15:13:33.416000+00:00,None,1,game_dc7af284-a056-4a87-bfe0-bd20f87fada9.json
904,dc7af284-a056-4a87-bfe0-bd20f87fada9,10,2,C,GREEN,2025-02-11 15:13:35.602000+00:00,None,1,game_dc7af284-a056-4a87-bfe0-bd20f87fada9.json
905,dc7af284-a056-4a87-bfe0-bd20f87fada9,11,0,A,GREEN,2025-02-11 15:13:37.179000+00:00,None,1,game_dc7af284-a056-4a87-bfe0-bd20f87fada9.json
906,d6cc47bf-a62d-4e8a-9a3e-67f55a481ce6,0,3,D,GREEN,2025-02-14 14:52:22.987000+00:00,None,1,game_d6cc47bf-a62d-4e8a-9a3e-67f55a481ce6.json


Games DataFrame:


,game_id,start_time,completion_time,total_duration,success,chosen_quadrant,correct,score,biased_quadrant,source_file
16,c8c3b5bb-68cf-4657-807c-7590a0579954,2025-02-13 13:32:26.384222+00:00,2025-02-13 13:45:55.615268+00:00,809.231046,False,3,False,-100,0,game_c8c3b5bb-68cf-4657-807c-7590a0579954.json
34,f863e848-387d-40e1-a4fa-9064e76bfc4d,2025-02-14 20:39:47.852554+00:00,2025-02-14 20:43:56.132081+00:00,248.279527,False,2,False,-100,0,game_f863e848-387d-40e1-a4fa-9064e76bfc4d.json
117,b12cf7e4-8125-4e4c-b038-923fa3c1e645,2025-02-13 13:27:26.931351+00:00,2025-02-13 13:31:30.329723+00:00,243.398372,True,2,True,100,2,game_b12cf7e4-8125-4e4c-b038-923fa3c1e645.json
9,f20d9df6-966b-42c9-9564-5a227e447a0e,2025-02-13 17:08:53.135000+00:00,2025-02-13 17:12:22.773551+00:00,209.638551,True,0,True,100,0,game_f20d9df6-966b-42c9-9564-5a227e447a0e.json
110,91d38a41-a495-4e58-9d00-1de2e86b6d43,2025-02-13 13:32:04.375602+00:00,2025-02-13 13:34:42.449000+00:00,158.073398,True,3,True,100,3,game_91d38a41-a495-4e58-9d00-1de2e86b6d43.json
...,...,...,...,...,...,...,...,...,...,...
6,d69294e8-731e-447f-9f4a-c3e80a08b56d,2025-02-13 14:35:32.302014+00:00,2025-02-13 14:35:38.365003+00:00,6.062989,True,1,True,100,1,game_d69294e8-731e-447f-9f4a-c3e80a08b56d.json
105,9896a898-ac4d-46b7-94db-7192058c50ff,2025-02-10 17:24:38.261393+00:00,2025-02-10 17:24:44.254338+00:00,5.992945,False,1,False,-100,0,game_9896a898-ac4d-46b7-94db-7192058c50ff.json
12,59be0148-38f2-44e5-a7c6-24614d0d6c8a,2025-02-13 13:31:33.182401+00:00,2025-02-13 13:31:38.009506+00:00,4.827105,False,2,False,-100,3,game_59be0148-38f2-44e5-a7c6-24614d0d6c8a.json
94,9b377a61-f5f5-476c-8dfc-8c5073f33db2,2025-02-13 13:24:25.341565+00:00,2025-02-13 13:24:29.797917+00:00,4.456352,False,2,False,-100,3,game_9b377a61-f5f5-476c-8dfc-8c5073f33db2.json
